# Top Down Table Based Parsing
#### Authors: Can Birol, Ryan Lambert, Sandip Sidhu 

This document will discuss the methods for parsing code and more generally grammars in a top down table based method. So what is top down table based parsing? To explain there are two key parts here, top down and table based. Top down parsing is analysing an input in a left to right way generating a leftmost derivation by expanding the given formal grammar rules. Table based refers to how the data structure is formed, specifically in this case a stack and transition table is used. This is in contrast to recursive descent parsers which have implicit stacks created by the recursive structure. 

For this document the focus will be on LL(k) parsers, more specifically LL(1) parsing. The reason for this because this document is intended to be an introductory manual for how to do top down parsing and LL(1) grammars are relatively easy to construct and understand. Table based parsing also has the advantage of being able to show the intermediate steps that occur from the input to the output. This can show users and help them understand how a language can be parsed. 

## Glossary of Terms

Below are some terms that will be used throughout the notebook that we thought it would be helpful to briefly define before going into detail.

**Parse:** To parse, or parsing, in English, is analyzing a sentence, dividing it into it's parts and identifying each of it's parts and their relation to each other. In the context of computer science, parsing refers to analyzing strings of symbols and checking them against the rules of a grammar.

**Bottom-up parsing:** Refers to how the parser recognizes lowest level details first and works it's way up to the highest level details. 

**Top-down parsing:** Inversely to bottom-up, top-down parsers recognize the highest level details first and works it's way down to the lowest level details.

**Grammar:** A grammar is simply the set of production rules that a string must follow.

**Context-free grammar:** Is a grammar that includes all possible strings in a given language.

**Lookahead:** Refers to how far in advance the parser can see in order to determine which rule to use. In the context of parsers it is usually represented by the k value.

**Backtracking:** Refers to the parsers ability to reverse and look at another option. For example, you're in a maze and reach a dead end, so you reverse and take the other available path."

**FIRST and FOLLOW:** FIRST refers to the set of terminals that can appear in the first position of any string. Whereas FOLLOW is the union between a FIRST and any non-terminal that follows it.

## History of Top Down Table Parsers

LR parsing was invented by Donald Knuth in 1965. Knuth proved that LR(k) grammars can be parsed with an execution time essentially proportional to the length of the program, and that every LR(k) grammar for k > 1 can be mechanically transformed into an LR(1) grammar for the same language. LR, standing for **L**eft to right with **R**ightmost derivation, was named as such because of the way it builds the parse tree, reading from Left to right, and then parsing bottom up with a rightmost derivation. The k value is the "lookahead" value which references the parsers ability to see input symbols ahead of the current one in order to decide how it is to be parsed. The k value is usually 1.

In 1969, Frank DeRemer invented the first subset of LR parsers, the LALR Parser or Look-Ahead LR Parser in his PhD paper "Practical Translators for LR(k) languages". In the paper it showed that if you have a language that is recognized by both an LR(0) parser and an LALR parser, the LALR parser will require the same amount of states as the LR(0) parser however the LALR parser will have more language recognition power. It was also found that the LALR parser was more memory efficient which makes it powerful enough for modern programming languages like Java. The LALR parser also has a k value. Like the LA parser, the k value in LALR(k) also represents the "lookahead". There is also a common LALR parser with a k value of two, meaning a two-token lookahead (LALR(2)). Both the LR parsers and LALR parsers can have a indefinite k-token lookup value, however these are rare as they are hard to implement. Later on in 1979 Frank DeRemer and Tom Pennello significantly improved the memory efficiency with a series of optimizations.

However both the LR parser and the LALR parser are bottom up parsers. Though the innovation of the LR parser paved the way for top down parsers to come. Introducing the LL parser.

The idea of LL(1) grammars was introduced by Lewis and Stearns in 1968 in their paper "Syntax directed transduction". LL parsers also use the k "lookahead" value like the LR and LALR parsers do. LL parsers parse from left to right, however, they parse from top to bottom instead of vise versa. Thus the name "top down parser". LL parsers also do this without backtracking at all which makes them very valuable as they are much easier to construct than their bottom up counterparts. Though the one downside is that LL parsers cannot recognize all context free languages.

## First example

In this example we will be parsing a very simple grammar. This grammar is essentially the character a, b, and c and the ability for them to be added within parenthesis. Full production rules below: 
1. S -> F
2. S -> (S + F)
3. F -> a
4. F -> b
5. F -> c

First we will generate for ourselves a parse table based on this grammar with the terminals as columns and non terminals as rows:

|      | ( | ) | a | b | c | + | $ |invalid|
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|   S  |   1  |   -1 |   0  |  0   |  0   |  -1  |   -1 |   -1 |
|  F   |  -1  | -1   |  2   |   3  |   4  |  -1  |  -1  | -1   |

Now we can code this example as shown below:



In [1]:
import time
#All constants are indexed from 0
sleeptime = 2

Term = 0
Rule = 1

# Terminals
T_LPAR = 0
T_RPAR = 1
T_A = 2
T_B = 3
T_C = 4
T_PLUS = 5
T_END = 6
T_INVALID = 7

# Non-terminals
N_S = 0
N_F = 1

#parse table
table = [[ 1, -1, 0, 0, 0, -1, -1, -1],
         [-1, -1, 2, 3, 4, -1, -1, -1]]

rules = [[(Rule,N_F)],
         [(Term,T_LPAR), (Rule,N_S), (Term,T_PLUS), (Rule,N_F), (Term,T_RPAR)],
         [(Term,T_A)], 
         [(Term,T_B)],
         [(Term,T_C)]]

stack = [(Term,T_END), (Rule,N_S)]

def tokenize(inputstring):
    print('Tokenize')
    tokens = []
    for c in inputstring:
        if c   == '+': tokens.append(T_PLUS)
        elif c == '(': tokens.append(T_LPAR)
        elif c == ')': tokens.append(T_RPAR)
        elif c == 'a': tokens.append(T_A)
        elif c == 'b': tokens.append(T_B)
        elif c == 'c': tokens.append(T_C)
        else: tokens.append(T_INVALID)
    tokens.append(T_END)
    print(tokens)
    return tokens

def TopDownTableParse(tokens):
    print('Parse')
    position = 0
    while len(stack) > 0:
        (stype, svalue) = stack.pop()
        token = tokens[position]
        if stype == Term:        
            if svalue == token:
                position += 1
                print('Pop', svalue)
                if token == T_END:
                    print('Input accepted')
                    break
            else:
                print('Bad term on input:', token)
                break
        elif stype == Rule:
            print('Current svalue', svalue, 'and token', token)
            rule = table[svalue][token]
            print('Applying rule', rule)
            for r in reversed(rules[rule]):
                stack.append(r)
        print('Current stack', stack)
        time.sleep(sleeptime)


In [2]:
inputstring = '(a+c)'
TopDownTableParse(tokenize(inputstring))

Tokenize
[0, 2, 5, 4, 1, 6]
Parse
Current svalue 0 and token 0
Applying rule 1
Current stack [(0, 6), (0, 1), (1, 1), (0, 5), (1, 0), (0, 0)]
Pop 0
Current stack [(0, 6), (0, 1), (1, 1), (0, 5), (1, 0)]
Current svalue 0 and token 2
Applying rule 0
Current stack [(0, 6), (0, 1), (1, 1), (0, 5), (1, 1)]
Current svalue 1 and token 2
Applying rule 2
Current stack [(0, 6), (0, 1), (1, 1), (0, 5), (0, 2)]
Pop 2
Current stack [(0, 6), (0, 1), (1, 1), (0, 5)]
Pop 5
Current stack [(0, 6), (0, 1), (1, 1)]
Current svalue 1 and token 4
Applying rule 4
Current stack [(0, 6), (0, 1), (0, 4)]
Pop 4
Current stack [(0, 6), (0, 1)]
Pop 1
Current stack [(0, 6)]
Pop 6
Input accepted


## Output analysis
The parser performs three types of steps depending on whether the top of the stack is a nonterminal, a terminal or the special symbol \$:

- If the top is a nonterminal (this section is represented in the elif stype == Rule branch) then it looks up in the parsing table on the basis of this nonterminal (represented by svalue) and the symbol on the input stream (represented by token) which rule of the grammar it should use to replace it with on the stack. If the parsing table indicates that there is no such rule then it reports an error and stops.
- If the top is a terminal (represented by the if stype == Term branch) then it compares it to the symbol on the input stream and if they are equal they are both removed. If they are not equal the parser reports an error and stops.
- If the top is \$ and on the input stream there is also a \$ (this is represented by the sub-branch if token == T_END) then the parser reports that it has successfully parsed the input, otherwise it reports an error. In both cases the parser will stop.

These steps are repeated until the parser stops, and then it will have either completely parsed the input or it will have reported an error.

## Formal Definition of an LL(1) Grammar
In the most basic sense, grammars that are context free and whose parsing table has no multiple entries in it are considered LL(1). LL(1) grammars are non ambiguous and also non left-recursive.

For example, if we have a grammar that is context free defined as **G = (V<sub>T</sub>,V<sub>N</sub>,S,P)**, and for every nonterminal and every string **a**, **b** where **a** =/= **b** and **A** -> **a** | **b**, two rules must also be true in order for the grammar to be LL(1):

**Rule 1:** FIRST(**a**) **intersect** FIRST(**b**) = **empty set** 

**Rule 2:** if **a** => **epsilon** then FIRST(**b**) **intersect** FOLLOW(**A**) = **empty set**

If these two rules remain true for your grammar, then it is LL(1).

*note: jupyter would not allow the insertion of greek and set theory characters. Therefore they were replaced with english characters and words

## Steps to create LL1 parsing table

#### Step 1 Remove alternations:

A_1 = X_1 X_2 X_3...X_n_1a | X_1 X_2 X_3...X_n_1b    

                                                   
A_2 = X_1 X_2 X_3...X_n_2  


A_3 = X_1 X_2 X_3...X_n_3a  | X_1 X_2 X_3...X_n_3b  
           
           . . .
           
A_k = X_1 X_2 X_3...X_n_k

#### The altered grammar looks like this:


A_1 = X_1 X_2 X_3...X_n_1a

A_1 = X_1 X_2 X_3...X_n_1b 

A_2 = X_1 X_2 X_3...X_n_2  

A_3 = X_1 X_2 X_3...X_n_3a 

A_3 = X_1 X_2 X_3...X_n_3b

           . . .
           
A_k = X_1 X_2 X_3...X_n_k

### Step 2 create FIRST sets for the right-hand sides of each of the grammarâs productions:

A_k = X_1 X_2 X_3...X_n_k
###### FIRST(X) for arbitrary x, we start by defining FIRST(X), for a single symbol X (a terminal, a nonterminal, or Î±):
##### Put FIRST(X1) â {Îµ} into FIRST(X). 
##### If Îµ is in FIRST(X1), then put F IRST(X2) â {Îµ} into F IRST(X). â If Îµ is in F IRST(X2), then put F IRST(X3) â {Îµ} into FIRST(X). â . . . â If Îµ is in FIRST(Xi) for 1  i  k (all production right- hand sides) then put Îµ into F IRST(X).

FIRST(A_1)=

FIRST(A_2)=

FIRST(A_3)=

    . . . 

FIRST(A_k)=

### Step 3 create Follow sets for the right-hand sides of each of the grammarâs productions:

A_k = X_1 X_2 X_3...X_n_k

#### For each production X â Î±AÎ², put FIRST(Î²) â {Ç«} in FOLLOW(A)
If Ç« is in FIRST(Î²) then put FOLLOW(X) into FOLLOW(A)
For each production X â Î±A, put FOLLOW(X) into FOLLOW(A)

FOLLOW(A_1)=

FOLLOW(A_2)=

FOLLOW(A_3)=

     . . .

FOLLOW(A_k)=

### Step 4 create Follow sets for the right-hand sides of each of the grammarâs productions:

for each production X â Î±

â for each terminal t in First(Î±): 
        put Î± in Table[X,t]

â if Î± is in First(Î±) then:
       for each terminal t in Follow(X): put Î± in Table[X,t]


|      | X_1 | X_2 | . | . | . | . |X_n_k|
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|  A_1  |   P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  A_2   |   P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  A_3   |    P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  .   |    P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  .   |    P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  .  |    P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |
|  A_K  |    P_1  |  P_2 |  .  |.   | .   |  .  |   P_k |


#### Steps to create Parsing Table 
A -> BC

C -> +BC | Îµ

B -> DF

F -> *DF | Îµ

D -> (E) | id

Step 1: Take grammar and remove alterations

A -> BC

C -> +BC

C -> Îµ

B -> DF

F -> *DF

F -> Îµ

D -> (A)

D -> id

Step 2:

FIRST(A)={(,id}

FIRST(C)={+,Îµ}

FIRST(B)={(,id}

FIRST(F)={*,Îµ}

FIRST(D)={(,id}

step 3:

FOLLOW(A)={$,)}

FOLLOW(C)={$,)}

FOLLOW(B)={+,$,)}

FOLLOW(F)={+,$,)}

FOLLOW(D)={*,+,$,)}

step 4:

|      | id | + | * | ( | ) | \$ |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|  A  | A -> BC  |  | A -> BC |  |  |    |
| C  |  | C -> +BC|    |   | F -> Îµ   |    |   
|  B   |B -> DF |  |   |B -> DF   |    |    | 
| F |    |  F -> Îµ|  B -> DF  |  |  F -> Îµ | F -> Îµ  |  
|  D  |    D -> id  |   |    |D -> (A)   |  |    | |  


## Limitations of top down table parsers

In order for a CFG to be an LL(1) grammar, certain conflicts must not arise, which we describe in this section.

Let A be a non-terminal. Then FIRST(A) is the set of terminals that can appear in the first position of any string derived from A. FOLLOW(A) is the union over FIRST(B) where B is any non-terminal that immediately follows A in the right hand side of a production rule.

There are two main types of conflicts First/First and First/Follow. These conflicts occurs as a result of ambiguity in the rule to choose. There needs to be a single choice per entry in the table. If there are more, the program does not know what path to take, as a result it is not parsable by LL(1):

#### FIRST/FIRST
The FIRST sets of two different grammar rules for the same non-terminal intersect. That is that the first position of a string derived from A is also the first position of a string derived from B. 

Example of First/First error producing grammar:
1. S -> E | E 'a'
2. E -> 'b' | Îµ

In [ ]:
import time
print('S -> E ')
print('S -> E ''\'a\'')
print('E -> ''\'b\'')
print('E -> Îµ')
time.sleep(1)
print("FIRST(S)=FIRST(E)")
time.sleep(1)
print("FIRST(E)={'b', Îµ}")
time.sleep(1)
print("FIRST(S)={E 'a'}")
time.sleep(1)
print("FIRST(E 'a')={'b',a}")

In [ ]:
from IPython.display import *
# here we define e to be Îµ
def ani():
    counter=1
    rule1='S -> E '
    rule2='S -> E ''\'a\''
    rule3='E -> ''\'b\''
    rule4='E -> e'
    for i in range(3):
        clear_output()
        if i==0:
            
            data=[
                 [' ','a','b','e'],
                 ['E','' ,'',''],
                 ['S','',''  ,''],
                 ]
    

        elif i==1:
            data = [
                 [' ','a','b','e'],
                 ['E',rule4 ,rule3,rule4],
                 ['S','','','']
                 ]

              
        elif i==2:
            
            data = [ [' ','a','b','e'],['E',rule4 ,rule3,rule4],['S',rule2,rule1 +' or '+ rule2 ,rule1]]
        display(HTML('<table><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data))))
        time.sleep(2.5)
ani()

#### FIRST/FOLLOW
The FIRST and FOLLOW set of a grammar rule overlap. That is an element in the FIRST(A) exists in FOLLOW(A) creating a conflict.

First/Follow Error Example Grammar:
1. S-> A 'a' 'b'
2. A-> 'a' | Îµ

In [ ]:
print('FIRST(S)=A ')
print('FOLLOW(S)=$')
time.sleep(1)
print('FIRST(A)={a,Îµ}')
print('FOLLOW(A)={a}')

In [ ]:
from IPython.display import *
# here we define e to be Îµ
def ani():

    rule1='S-> A \'a\' \'b\''
    rule2='A -> \'a\''
    rule3='A -> e'
    for i in range(2):
        clear_output()
        if i==0:
            
            data=[
                 [' ','a','b','e'],
                 ['A','' ,'',''],
                 ['S','',''  ,''],
                 ]
    

        elif i==1:
            data = [
                 [' ','a','b','e'],
                 ['E',rule1+" or "+rule2 ,"",''],
                 ['S','','','']
                 ]

        display(HTML('<table><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data))))
        time.sleep(2.5)
ani()

## More Complex Example

In this example we will construct a LL(1) parser for the following grammar:
1. S -> S + T | S - T | S + C | S - C | T | C
2. T -> a | b | c
3. C -> 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9

This grammar essentially allows for addition and subtraction of single digit constants or up to three variables called a, b, and c. This grammar has an initial First/Follow and First/First conflict due to the left recursion so we need to remove that first. We so an example of this below: 
1. S -> T E | C E
2. E -> + T E | - T E | + C E| - C E | e 
3. T -> a | b | c
4. C -> 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9

However this is not good enough as the second rule still has a First/First conflict so we will alter the grammar again as shown below:
1. S -> F E
2. E -> + F E 
3. E -> - F E
4. F -> T 
5. F -> C
6. T -> a
7. T -> b
8. T -> c
9. C -> 0
10. C -> 1
11. C -> 2
12. C -> 3
13. C -> 4
14. C -> 5
15. C -> 6
16. C -> 7
17. C -> 8
18. C -> 9
19. E -> Empty


Now this grammar is conflict free so we can construct a table for it:

|      | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | a    | b    | c    | +    | -    | \$   |invalid|
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|  S   | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | 1    | -1   | -1   | -1   | -1   |
|  E   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | 2    | 3    | 19   | -1   |
|  F   | 5    | 5    | 5    | 5    | 5    | 5    | 5    | 5    | 5    | 5    | 4    | 4    | 4    | -1   | -1   | -1   | -1   |
|  T   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | -1   | 6    | 7    | 8    | -1   | -1   | -1   | -1   |
|  C   | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   | 18   | -1   | -1   | -1   | -1   | -1   | -1   | -1   |

And now we will construct the parser as shown below:

In [ ]:
import time
#All constants are indexed from 0
sleeptime = 2

Term = 0
Rule = 1

# Terminals
#The constants 0-9 are given the corresponding value, this is implicit in the way that the program works
T_A = 10
T_B = 11
T_C = 12
T_PLUS = 13
T_MINUS = 14
T_END = 15
T_INVALID = 16

# Non-terminals
N_S = 0
N_E = 1
N_F = 2
N_T = 3
N_C = 4

#parse table
table = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 2, 18, -1],
         [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5, 6, 7, -1, -1, -1, -1],
         [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, -1, -1, -1, -1, -1, -1, -1]]

rules = [[(Rule,N_F), (Rule, N_E)],
         [(Term,T_PLUS), (Rule,N_F), (Rule,N_E)],
         [(Term,T_MINUS), (Rule,N_F), (Rule,N_E)],
         [(Rule, N_T)],
         [(Rule, N_C)],
         [(Term,T_A)], 
         [(Term,T_B)],
         [(Term,T_C)], 
         [(Term,0)], 
         [(Term,1)], 
         [(Term,2)], 
         [(Term,3)], 
         [(Term,4)], 
         [(Term,5)], 
         [(Term,6)], 
         [(Term,7)], 
         [(Term,8)], 
         [(Term,9)],
         []]

stack = [(Term,T_END), (Rule,N_S)]

def tokenizeComplex(inputstring):
    print('Tokenize')
    tokens = []
    for c in inputstring:
        if c   == '+': tokens.append(T_PLUS)
        elif c == '-': tokens.append(T_MINUS)
        elif c == 'a': tokens.append(T_A)
        elif c == 'b': tokens.append(T_B)
        elif c == 'c': tokens.append(T_C)
        elif c == '0': tokens.append(0)
        elif c == '1': tokens.append(1)
        elif c == '2': tokens.append(2)
        elif c == '3': tokens.append(3)
        elif c == '4': tokens.append(4)
        elif c == '5': tokens.append(5)
        elif c == '6': tokens.append(6)
        elif c == '7': tokens.append(7)
        elif c == '8': tokens.append(8)
        elif c == '9': tokens.append(9)
        else: tokens.append(T_INVALID)
    tokens.append(T_END)
    print(tokens)
    return tokens

def topDownTableParseComplex(tokens):
    print('Parse')
    position = 0
    print('Current stack', stack)
    while len(stack) > 0:
        (stype, svalue) = stack.pop()
        token = tokens[position]
        if stype == Term:        
            if svalue == token:
                position += 1
                print('Pop', svalue)
                if token == T_END:
                    print('Input accepted')
                    break
            else:
                print('Bad term on input:', token)
                break
        elif stype == Rule:
            print('Current svalue', svalue, 'and token', token)
            rule = table[svalue][token]
            print('Applying rule', rule)
            for r in reversed(rules[rule]):
                stack.append(r)
        print('Current stack', stack)
        time.sleep(sleeptime)


In [ ]:
inputstring = '4+a-c'
topDownTableParseComplex(tokenizeComplex(inputstring)) 

## LL(1) Parser Parser

In this section a parser for grammars in BNF form will be described, then implemented in python below. First a definition of symbols used in our BNF to be read. All non-terminal symbols must be enclosed in <>, terminals must be enclosed in single quotations ' ', ::= is the production symbol, | is used for alteration and ; for EoL. There are two reserved characters \$ and e, to represent EoF and epsilon respectively (changing which character is reserved for epsilon is possible by changing the epsilon variable at the top of the program). Ex. the first example in our BNF would look like:
0. < S > ::= < F >;
1. < S > ::= '(' < S > '+' < F > ')';
2. < F > ::= 'a';
3. < F > ::= 'b';
4. < F > ::= 'c';

note: there is very limited error handling so grammars that are improperly formatted will create errors

In [6]:
T_ = []
N_ = []
epsilon = 'e'
first = [[]]
rules = [[]]
follow = [[]]

def grammarParser(inputString):
    global T_
    global N_
    global first
    global rules
    global follow
    global epsilon
    printTable = [[]]
    table = [[]]
    
    inputString = ''.join(inputString.split()) #this strips the inputString of whitespace
    
    if initNandT(inputString) == -1:
        return
    T_.append("$")
    T_.append("Invalid")
    
    #initialize the printTable for human output and the table which will store the result
    w, h = len(T_), len(N_)
    printTable = [[-1] * (w+1) for n in range(h+1)]
    printTable[0][0] = " "
    table = [[-1] * (w) for m in range(h)]
    i = 1
    for t in T_:
        printTable[0][i] = t
        i += 1
    i = 1
    
    for n in N_:
        printTable[i][0] = n
        i+=1
    
    print("Initial Human Readable table: ")
    for s in printTable:
        print(s)
    
    #initialize the rules list
    ruleStrings = inputString[:len(inputString)-1].split(";") #this is hardcoded this way because a grammar should end with a ;
    
    w, h = 2, 0
    for s in ruleStrings:
        h += s.count("|")
    h += len(ruleStrings)
    rules = [[""] * (w) for i in range(h)]
    #this section reformats alterations into new rules
    index = 0
    for s in ruleStrings:
        abi = s.find(">") #angle bracket index
        ruleChar = s[1:abi]
        s = s[s.find("=")+1:]  
        barIndex = s.find("|")
        if barIndex != -1: #begin processing alterations
            tempS = s.split("|")
            for x in tempS:
                rules[index][0] = ruleChar
                production = helper(x)
                rules[index][1] = production
                index+=1
        else: #this handles adding rules with no alterations
            rules[index][0] = ruleChar
            production = helper(s)
            rules[index][1] = production
            index +=1
    print("Rules array: \n", rules)
    
    #now we will compute the FIRST and FOLLOW sets
    w, h = 2, len(N_)
    first = [[""] * (w) for i in range(h)]
    follow = [[""] * (w) for i in range(h)]
    i = 0
    for n in N_:
        first[i][0] = n
        first[i][1] = []
        follow[i][0] = n
        follow[i][1] = []
        i+=1
        
    #compute first
    changing = 1 
    failsafe = 0
    while changing and (failsafe<10):
        changing = 0
        for rule in rules:
            index = N_.index(rule[0]) #this stores the index of the nonterminal lhs of the rule
            temp = first[index][1]
            first[index][1] = union(first[index][1], computefirst(rule[1]))
            if changingcheck(first[index][1], temp):
                changing = 1
        failsafe += 1
    
    print("The FIRST array: \n", first)    
    #compute follow
    follow[0][1] = ["$"] #start non terminal has EoF in its follow
    changing = 1
    failsafe = 0
    while changing and (failsafe<10):
        changing = 0
        for rule in rules:
            for x in rule[1]:
                if x in N_:   # the body of this loop is over all the nonterms on the rhs
                    nonterm = N_.index(x)
                    index = rule[1].index(x)
                    if index+1 < len(rule[1]):
                        comFir = computefirst(rule[1][index+1:])
                        temp = follow[nonterm][1]
                        follow[nonterm][1] = union(follow[nonterm][1], excludeEpsilon(comFir))
                        if epsilon in comFir:
                             follow[nonterm][1] = union(follow[nonterm][1], follow[N_.index(rule[0])][1])
                        if changingcheck(follow[nonterm][1], temp):
                            changing = 1
                    elif "$" not in follow[nonterm][1]:
                            follow[nonterm][1].append("$")
                            changing = 1
        failsafe += 1
    print("The FOLLOW array: \n", follow)
    
    i = 0 #i here will represent the rule's index value
    for rule in rules:
        lhs = rule[0]
        nonterm = N_.index(rule[0])
        rhs = rule[1]
        tmp = computefirst(rhs)
        if epsilon in tmp:
            tmp = union(tmp, follow[nonterm][1])
            tmp.remove(epsilon)
        for t in tmp:
            term = T_.index(t)
            if table[nonterm][term] != -1:
                print("Grammar is not LL(1)"); return -1
            table[nonterm][term] = i
            printTable[nonterm+1][term+1] = i
        i += 1
    
    print("Final Human Readable table: ")
    for s in printTable:
        print(s)
        
    return table
def computefirst(rhs):
    global first
    global epsilon
    tmp = []
    i = 0
    etest = True
    while i < len(rhs):
        if rhs[i] in T_:
            tmp.append(rhs[i])
            etest = False
            break
        else:
            index = N_.index(rhs[i]) #grab the index of the nonterminal found
            tmp = union(tmp, excludeEpsilon(first[index][1]))
            if epsilon not in first[index][1]:
                etest = False
                break
        i+=1
    if etest:
        tmp.extend(epsilon)
    return tmp
def excludeEpsilon(arr):
    global epsilon
    if epsilon in arr:
        arr.remove(epsilon)
    return arr
        
def changingcheck(a, b):
    for x in a:
        if x not in b:
            return True
    return False

def union(a, b):
    return list(set(a) | set(b))

def initNandT(inputString):
    global T_
    global N_
    T_ = []
    N_ = []
    currentID = ""
    i = 0;
    while i < len(inputString): 
        c = inputString[i]
        if c == '<':
            while 1:
                i +=1
                if (i < len(inputString)):
                    c = inputString[i]
                    if (c == '>'): 
                        break
                    else:
                        currentID += c
                else: 
                    print("reached EoF before closing >"); return -1
                
            if currentID not in N_:
                N_.append(currentID)
            currentID = ""
        elif c == '\'':
            while 1:
                i +=1
                if (i < len(inputString)):
                    c = inputString[i]
                    if (c == '\''): 
                        break
                    else:
                        currentID += c
                else: 
                    print("reached EoF before closing \'"); return -1
                
            if currentID not in T_:
                T_.append(currentID)
            currentID = ""
        i+=1
    return 0
def helper(s):
    
    i = 0
    returnl = []
    while i < len(s):
        temp3 = ""
        if s[i] == "<":
            while 1:
                i +=1
                if (s[i] == '>'): break #this is guaranteed now so extra checks are not needed
                temp3 = temp3 + s[i]
        elif s[i] == '\'':
            while 1:
                i +=1
                if (s[i] == '\''): break #this is guaranteed now so extra checks are not needed
                temp3 = temp3 + s[i]
        returnl.extend(temp3)
        i+=1
    return returnl

In [2]:
grammarParser("< S > ::= < F >; < S > ::= '(' < S > '+' < F > ')';< F > ::= 'a' | 'b' | 'c';")

Initial Human Readable table: 
[' ', '(', '+', ')', 'a', 'b', 'c', '$', 'Invalid']
['S', -1, -1, -1, -1, -1, -1, -1, -1]
['F', -1, -1, -1, -1, -1, -1, -1, -1]
Rules array: 
 [['S', ['F']], ['S', ['(', 'S', '+', 'F', ')']], ['F', ['a']], ['F', ['b']], ['F', ['c']]]
The FIRST array: 
 [['S', ['a', 'c', 'b', '(']], ['F', ['a', 'c', 'b']]]
The FOLLOW array: 
 [['S', ['+', '$']], ['F', [')', '$']]]
Final Human Readable table: 
[' ', '(', '+', ')', 'a', 'b', 'c', '$', 'Invalid']
['S', 1, -1, -1, 0, 0, 0, -1, -1]
['F', -1, -1, -1, 2, 3, 4, -1, -1]


[[1, -1, -1, 0, 0, 0, -1, -1], [-1, -1, -1, 2, 3, 4, -1, -1]]

In [7]:
#this is an example of an ambigous grammar that looks like it should be LL(1) but isn't
grammarParser("<S>::= <A> | <B>; <A>::= 'e' | 'a';<B>::=<A>'b'|'b';")

Initial Human Readable table: 
[' ', 'e', 'a', 'b', '$', 'Invalid']
['S', -1, -1, -1, -1, -1]
['A', -1, -1, -1, -1, -1]
['B', -1, -1, -1, -1, -1]
Rules array: 
 [['S', ['A']], ['S', ['B']], ['A', ['e']], ['A', ['a']], ['B', ['A', 'b']], ['B', ['b']]]
The FIRST array: 
 [['S', ['a', 'b']], ['A', ['a']], ['B', ['a', 'b']]]
The FOLLOW array: 
 [['S', ['$']], ['A', ['b', '$']], ['B', ['$']]]
Grammar is not LL(1)


-1

## Why table parsing has been replaced and what we can learn from it

This section will discuss the exact differences between LL(1) parsers and LALR(1) parsers. Specifically will discuss YACC and the ease of creating LALR(1) parsers and the difficulties with LL(1) regarding context. 

It is possible that LL(1) parsers and LALR(1) parsers are the same. If all symbols with empty derivations in the LL(1) grammar have non-empty derivations, the LL(1) and LALR(1) parsers are considered equal. The difficulty with generating LL(1) parsers as opposed to their LALR(1) counterparts is that there are some context free languages that are not recognized by the LL(1) that are by the LALR(1). This coupled with the fact that LALR parsers are more memory efficient gives them the edge. With the difficulty that top-down parsing has on analysis of semantics and the stark limitations of LL(k), LALR parsers have proven to be a little more versatile and easier to construct in nature.

YACC (stands for Yet Another Compiler Compiler) is a unix program that generates LALR (**L**ook **A**head **L**eft-to-Right **R**ight-derivation) parsers. It was created in the early 70's by Stephen C. Johnson. YACC became so popular that for a time it was the default parser generator on most UNIX based computers and became the basis for many parser generators to come in the future.

### Biblography 

Aho, Alfred V., "Compilers : principles, techniques, & tools", Pearson/Addison Wesley, 2007.

DeRemer, Frank; Pennello, Thomas; "Efficient Computation of LALR(1) Look-Ahead Sets", October 1982

Dick Grune, Cerial Jacobs, "Parsing Techniques", Springer, 2008.

Dick Grune, Kees van Reeuwijk, Henri E. Bal, Ceriel J.H. Jacobs, Koen Langendoen, "Modern Compiler Design", Springer, 2012.

Frost, R., Hafiz, R. and Callaghan, P. "Modular and Efficient Top-Down Parsing for Ambiguous Left-Recursive Grammars .", 2007

Aho, Alfred V., Ullman, Jeffrey D. "The Theory of Parsing, Translation, and Compiling" Englewood Cliffs, NJ: Prentice-Hall. 1972

Rosenkrantz, D. J., Stearns, R. E., "Properties of Deterministic Top Down Grammars", 1970